In [1]:
#import OpenCV module
import cv2
#import os module for reading training data directories and paths
import os
#import numpy to convert python lists to numpy arrays as 
#it is needed by OpenCV face recognizers
import numpy as np
import sys

#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["", "Sachin Tendulkar", "Elvis Presley", "Shahrukh Khan"]

#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')

    
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

#this function will read all persons' training images, detect face from each image
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    dirs = os.listdir(data_folder_path)
    
    
    faces = []
    labels = []
    
    #to go through each directory and read images within it
    for dir_name in dirs:
        
        
        if not dir_name.startswith("s"):
            continue;
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        label = int(dir_name.replace("s", ""))
        
        #build path of directory containin images for current subject subject
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #------STEP-3--------
        for image_name in subject_images_names:

            if image_name.startswith("."):
                continue;
            

            image_path = subject_dir_path + "/" + image_name

            #read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detect face
            face, rect = detect_face(image)
            
            #------STEP-4--------
            
            if face is not None:
                
                faces.append(face)                
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

#prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
#print("Preparing data...")
faces, labels = prepare_training_data("training-data")
#print("Data prepared")

#to print total faces and labels
#print("Total faces: ", len(faces))
#print("Total labels: ", len(labels))


face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

#function to draw rectangle on image 
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
    
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer
    label= face_recognizer.predict(face)
    label = list(label)
    #print(label)
    if (label[1]) >= 30.0:
        print ("Image not in dataset")
        sys.exit()
    
    
    
    
    #get name of respective label returned by face recognizer
    label_text = subjects[label[0]]
    name = label_text
    
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img, name

print("Predicting images...")
print (" ")

#load test images
test_img2 = cv2.imread("test-data/test3.jpg")     #   --------- give name of image you want want to test

#perform a prediction
predicted_img2, name2 = predict(test_img2)
#print("Prediction complete")
print ("Person recognized is: ", name2)
print (" ")

#display both images
cv2.imshow("Image", predicted_img2)
cv2.waitKey(1000)
cv2.destroyAllWindows()


# VEHICLE NUMBER PLATE 

import cv2
import random
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

images_dir = "data/images"
image_files = os.listdir(images_dir)

image_path = "{}/{}".format(images_dir, "car_2.png")    #   ------- give name of image you want want to test

image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def plot_images(img1, img2, title1="", title2=""):
    fig = plt.figure(figsize=[15,15])
    ax1 = fig.add_subplot(121)
    ax1.imshow(img1, cmap="gray")
    ax1.set(xticks=[], yticks=[], title=title1)

    ax2 = fig.add_subplot(122)
    ax2.imshow(img2, cmap="gray")
    ax2.set(xticks=[], yticks=[], title=title2)
    
blur = cv2.bilateralFilter(gray, 11,90, 90)

edges = cv2.Canny(blur, 30, 200)

cnts, new = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

image_copy = image.copy()

_ = cv2.drawContours(image_copy, cnts, -1, (255,0,255),2)

cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]

image_copy = image.copy()
_ = cv2.drawContours(image_copy, cnts, -1, (255,0,255),2)

plate = None
for c in cnts:
    perimeter = cv2.arcLength(c, True)
    edges_count = cv2.approxPolyDP(c, 0.02 * perimeter, True)
    if len(edges_count) == 4:
        x,y,w,h = cv2.boundingRect(c)
        plate = image[y:y+h, x:x+w]
        break

cv2.imwrite("plate.png", plate)

import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
text = pytesseract.image_to_string(plate, lang="eng")

print("Vehicle number is:", text)
print (" ")

# DATASET

import pandas as pd
data = pd.read_csv('result.csv')
print ("Dataset is: ")
print (data)
print (" ")

# Test => whether driver can enter or not

Name = list(data['Name'])
try:
    if name2 in Name:
        print("Name in dataset")
        
except:
    print("Face isn't recognized: ENTRY DENIED")
    sys.exit()
    
Text = list(data['Vehicle_No'])
try:
    if text in Text:
        print("Number plate in dataset")
        print (" ")
        
except:
    print("Number plate missing: ENTRY DENIED")
    sys.exit()

print ("position of person's name in data set is at")
print (Name.index(name2))

print ("position of vehicle number in dataset is at")
print (Name.index(name2))
print (" ")

if Name.index(name2) == Text.index(text):
    print("Number plate matches with the corresponding person:: ENTRY GRANTED")
else:
    print("Number plate does not matches with the corresponding person:: ENTRY DENIED")

Predicting images...
 
Person recognized is:  Sachin Tendulkar
 
Vehicle number is: DL4CCB1111
 
Dataset is: 
               Name  Vehicle_No         Result
0  Sachin Tendulkar     3JOH22A  You can enter
1     Elvis Presley  DL4CCB1111  You can enter
 
Name in dataset
Number plate in dataset
 
position of person's name in data set is at
0
position of vehicle number in dataset is at
0
 
Number plate does not matches with the corresponding person:: ENTRY DENIED
